In [ ]:
#Import required libraries
import cv2
import numpy as np
import sys
import scipy.stats as st
import os
import math
import glob
import random

In [ ]:
#Coding Q1 
#Referring the link:
#https://docs.opencv.org/master/dc/dbb/tutorial_py_calibration.html

image = cv2.imread('chessboard.jpg') #image to be read from folder 

# termination criteria
termination_criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
object_points = np.zeros((6*7,3), np.float32) 
object_points[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)


#converting image to grayscale
grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#Finds the positions of internal corners of the chessboard.
ret, corners = cv2.findChessboardCorners(grayscale_image, (7,6), None)

# If corners are found then it will proceeds further
if ret:
	#corner subpix refines the corner locations
	refineCorners=cv2.cornerSubPix(grayscale_image,corners, (11,11), (-1,-1), termination_criteria) #process of corner position refinement stops either after criteria.maxCount iterations or when the corner position moves by less than criteria.epsilon on some iteration.


	#Write into a file.
	#Reference on how to write to file: https://www.geeksforgeeks.org/reading-writing-text-files-python/

	file = open("collectpoints.txt", "w")
	for objp, cornerposition in zip(object_points, corners.reshape(-1,2)): #zip() function returns an iterator of tuples based on the iterable object
		file.write(str(objp[0]) + ' ' + str(objp[1]) + ' ' + str(objp[2]) + ' ' + str(cornerposition[0]) + ' ' + str(cornerposition[1]) + '\n')
	file.close()

In [ ]:
#Q2 : Camera Calibration and calculation of Mean Squared Error:
def calculateMSE(points3D, points2D, M):

    points2DH = []
    points2D = []
    points2D_ref = []

    MSEx = 0
    MSEy = 0

    f = open("3D_world_points.txt", "r")
    for line in f:
        points3DH = np.array([[float(line.split()[0]), float(line.split()[1]), float(line.split()[2]), 1.0]])
        a = (M*points3DH.T)
        points2DH.append(a)
    f.close()

    g = open("2D_camera_points.txt", "r")
    for line in g:
        points2D_ref.append([float(line.split()[0]), float(line.split()[1])])
    g.close()

    #making 3D world co-ordinates points homogeneous
    for point in points2DH:
        point[0] = point[0]/(point[2])
        point[1] = point[1]/(point[2])
        point[2] = point[2]/(point[2])
        point = np.delete(point, (2), axis = 0)
        points2D.append(point)

    for i in range(0, len(points2D_ref)):
        MSEx += (points2D_ref[i][0]-points2D[i][0])**2
        MSEy += (points2D_ref[i][1]-points2D[i][1])**2

    MSEx = MSEx/len(points2D_ref)
    MSEy = MSEy/len(points2D_ref)
    MSE = MSEx+MSEy

    return MSE


def projection_matrix_calculation(points3D, points2D):
    A = np.zeros((len(points3D)*2, 12))
    d = 0
    for i in range(0, len(points3D)):
        x = np.array([[points3D[i][0], points3D[i][1], points3D[i][2], points3D[i][3], 0, 0, 0, 0, -points2D[i][0]*points3D[i][0], -points2D[i][0]*points3D[i][1], -points2D[i][0]*points3D[i][2], -points2D[i][0]*points3D[i][3]]])
        y = np.array([[0, 0, 0, 0, points3D[i][0], points3D[i][1], points3D[i][2], points3D[i][3], -points2D[i][1]*points3D[i][0], -points2D[i][1]*points3D[i][1], -points2D[i][1]*points3D[i][2], -points2D[i][1]*points3D[i][3]]])
        A[d] = x
        A[d+1] = y
        d+= 2

    A_transpose_A = np.dot(A.T, A)

    #Reference: https://numpy.org/doc/stable/reference/generated/numpy.linalg.svd.html
    U, S, V = np.linalg.svd(A_transpose_A, full_matrices=True)
    result = V[:,11] 

    #Calculating Projection Matrix elements:
    a1 = np.array([result[0], result[1], result[2]])
    a2 = np.array([result[4], result[5], result[6]])
    a3 = np.array([result[8], result[9], result[10]])
    b = np.matrix([[result[3]], [result[7]], [result[11]]]) 

    ro = 1/np.linalg.norm(a3)
    u0 = ro**2*(np.dot(a1, a3))
    v0 = ro**2*(np.dot(a2, a3))
    alfav = math.sqrt(ro**2*np.dot(a2,a2)-v0**2)
    s = (ro**4/alfav)*(np.dot(np.cross(a1, a3), np.cross(a2, a3)))
    alfau = math.sqrt(ro**2*np.dot(a1, a1)-s**2-u0**2)

    #Calculating intrinsic parameter K_star and extrinsic parameters R_Star and T_star(Translation and Rotation.)
    K = np.matrix([[alfau, s, u0],[0.0, alfav, v0],[0.0, 0.0, 1]])
    r1 = np.cross(a2,a3)/np.linalg.norm(np.cross(a2,a3))
    r3 = a3
    r2 = np.cross(r3, r1)
    eps = np.sign(b[2][0])
    Kinverse = np.linalg.inv(K)
    T = (np.dot(Kinverse,b))
    T *= eps*ro 
    
    e_parameters = np.matrix([[r1[0], r1[1], r1[2], T[0][0]],[r2[0], r2[1], r2[2], T[1][0]],[r3[0], r3[1], r3[2], T[2][0]]])

    #Projection Matrix.
    M = np.dot(K, e_parameters)
 
    #print("Extrinsic Parameters: ", e_parameters)
    #print("Intrinsic Parameters: ", K)

    return M


def main():
    f = open("3D_world_points.txt", "r")
    g = open("2D_camera_points.txt", "r")
    points3D = []
    points2D = []

    for line in f:
        points3D.append([float(line.split()[0]), float(line.split()[1]), float(line.split()[2]), 1.0])
    f.close()

    for line in g:
        points2D.append([float(line.split()[0]), float(line.split()[1])])
    g.close()

    M = projection_matrix_calculation(points3D, points2D)
    Mean_squared_error = calculateMSE(points3D, points2D, M)

    print("Mean squared error", Mean_squared_error)

if __name__ == '__main__':
    main()

Mean squared error [[matrix([[156357.97663195]])]]


In [ ]:
def calculaparameters(points3D, points2D):

    A = np.zeros((len(points3D)*2, 12))

    j = 0
    for i in range(0, len(points3D)):
        x = np.array([[points3D[i][0], points3D[i][1], points3D[i][2], points3D[i][3], 0, 0, 0, 0, -points2D[i][0]*points3D[i][0], -points2D[i][0]*points3D[i][1], -points2D[i][0]*points3D[i][2], -points2D[i][0]*points3D[i][3]]])
        y = np.array([[0, 0, 0, 0, points3D[i][0], points3D[i][1], points3D[i][2], points3D[i][3], -points2D[i][1]*points3D[i][0], -points2D[i][1]*points3D[i][1], -points2D[i][1]*points3D[i][2], -points2D[i][1]*points3D[i][3]]])
        A[j] = x
        A[j+1] = y
        j+= 2

    ATA = np.dot(A.T, A)
    U, S, V = np.linalg.svd(ATA, full_matrices=True)
    sol = V[:,11]

    a1 = np.array([sol[0], sol[1], sol[2]])
    a2 = np.array([sol[4], sol[5], sol[6]])
    a3 = np.array([sol[8], sol[9], sol[10]])
    b = np.matrix([[sol[3]], [sol[7]], [sol[11]]])

    ro = 1/np.linalg.norm(a3)
    u0 = ro**2*(np.dot(a1, a3))
    v0 = ro**2*(np.dot(a2, a3))
    alfav = math.sqrt(ro**2*np.dot(a2,a2)-v0**2)
    s = (ro**4/alfav)*(np.dot(np.cross(a1, a3), np.cross(a2, a3)))
    alfau = math.sqrt(ro**2*np.dot(a1, a1)-s**2-u0**2)

    K = np.matrix([[alfau, s, u0],[0.0, alfav, v0],[0.0, 0.0, 1]])

    r1 = np.cross(a2,a3)/np.linalg.norm(np.cross(a2,a3))
    r3 = a3
    r2 = np.cross(r3, r1)
    eps = np.sign(b[2][0])
    Kinverse = np.linalg.inv(K)
    T = (np.dot(Kinverse,b))
    T *= eps*ro
    e_parameters = np.matrix([[r1[0], r1[1], r1[2], T[0][0]],[r2[0], r2[1], r2[2], T[1][0]],[r3[0], r3[1], r3[2], T[2][0]]])

    print("Extrinsic parameters", e_parameters)
    print("Intrinsic parameters", K)

    M = np.dot(K, e_parameters)

    return M

def calculateMSE(points3D, points2D, M):

    points2DH = []
    points2D = []
    points2D_ref = []

    MSEx = 0
    MSEy = 0

    f = open("3D_world_points.txt", "r")
    for line in f:
        pointsDH3 = np.array([[float(line.split()[0]), float(line.split()[1]), float(line.split()[2]), 1.0]])
        a = (M*pointsDH3.T)
        points2DH.append(a)
    f.close()

    g = open("2D_camera_points.txt", "r")
    for line in g:
        points2D_ref.append([float(line.split()[0]), float(line.split()[1])])
    g.close()

    for point in points2DH:
        point[0] = point[0]/(point[2])
        point[1] = point[1]/(point[2])
        point[2] = point[2]/(point[2])
        point = np.delete(point, (2), axis = 0)
        points2D.append(point)

    for i in range(0, len(points2D_ref)):
        MSEx += (points2D_ref[i][0]-points2D[i][0])**2
        MSEy += (points2D_ref[i][1]-points2D[i][1])**2

    MSEx = MSEx/len(points2D_ref)
    MSEy = MSEy/len(points2D_ref)
    MSE = MSEx+MSEy

    return MSE


def RANSAC(points3D, points2D, n, d, k):
    Ms = []
    MSEs= []
    inliers3D = []
    inliers2D = []

    for i in range (0, k):
        points3D_random = []
        points2D_random = []
        distances = []

        for i in range(0, n):
            random.seed(57)
            random_int = random.randint(0, len(points3D)-1)
            points3D_random.append(points3D[random_int])
            points2D_random.append(points2D[random_int])

        M = calculaparameters(points3D_random, points2D_random)

        for i in range(0, len(points3D_random)):
            distance = math.sqrt(calculateMSE(points3D_random[i], points2D_random[i], M))
            distances.append(distance)
        distances.sort()
        if len(distances)%2 == 0:
            median = (distances[int(len(distances)/2)]+distances[(int(len(distances)/2)-1)])/2
        else:
            median = distances[int((len(distances)/2)-0.5)]
        t = 1.5*median

        for i in range(0, len(points3D)):
            distance = math.sqrt(calculateMSE(points3D[i], points2D[i], M))
            if distance < t:
                inliers3D.append(points3D[i])
                inliers2D.append(points2D[i])

        if len(inliers3D) >= d:
            M = calculaparameters(inliers3D, inliers2D)
            MSE = calculateMSE(inliers3D, inliers2D, M)
            print("Projection Matrix", M)
            print("Mean squared Error", MSE)

        Ms.append(M)
        MSEs.append(MSE)

    print("Ms", Ms)
    print("MSEs", MSEs)
    MSEmin = MSE[0]
    Mdef = M[0]
    for i in range(0, len(MSEs)):
        if MSEs[i] < MSEmin:
            MSEmin = MSE[i]
            Mdef = Ms[i]

    return Mdef, MSEmin


def main():
    f = open("3D_world_points.txt", "r")
    g = open("2D_camera_points.txt", "r")
    points3D = []
    points2D = []

    for line in f:
        points3D.append([float(line.split()[0]), float(line.split()[1]), float(line.split()[2]), 1.0])
    f.close()

    for line in g:
        points2D.append([float(line.split()[0]), float(line.split()[1])])
    g.close()

    Mdef, MSEmin = RANSAC(points3D, points2D, 9, 7, 4)

    print("Projection matrix from Ransac", Mdef)
    print("Mean squared error from RANSAC", MSEmin)

if __name__ == '__main__':
    main()

Extrinsic parameters [[0.43027617379120076 -0.2518853361564332 0.8668426568281957
  matrix([[-59.98360985]])]
 [0.0012046325082672444 -0.0007051965290476988 -0.000802860041294889
  matrix([[-0.04328371]])]
 [0.0008135231042138601 0.0013896783906262018 3.829524793794086e-16
  matrix([[4.78591357e-17]])]]
Intrinsic parameters [[5.15603874e+00 6.55943243e-02 1.53818789e+00]
 [0.00000000e+00 2.62590717e+02 4.69981157e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Extrinsic parameters [[-0.03776722191811869 0.10510573798575978 0.9937436393713703
  matrix([[56.59470397]])]
 [-0.02479749170392626 0.06256255994484008 -0.007559511435190444
  matrix([[0.18926868]])]
 [-0.0629656940361824 -0.024927851399106443 0.00024354458053454573
  matrix([[0.03701604]])]]
Intrinsic parameters [[ 2.62897477e-03  2.69915877e-03 -5.84031276e-02]
 [ 0.00000000e+00  1.25400751e+01 -7.44089503e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Projection Matrix [[0.0035111720253158587 0.001901051102976

IndexError: ignored

In [ ]:
# Testing on Noisy data.
def calculaparameters(points3D, points2D):

    A = np.zeros((len(points3D)*2, 12))

    j = 0
    for i in range(0, len(points3D)):
        x = np.array([[points3D[i][0], points3D[i][1], points3D[i][2], points3D[i][3], 0, 0, 0, 0, -points2D[i][0]*points3D[i][0], -points2D[i][0]*points3D[i][1], -points2D[i][0]*points3D[i][2], -points2D[i][0]*points3D[i][3]]])
        y = np.array([[0, 0, 0, 0, points3D[i][0], points3D[i][1], points3D[i][2], points3D[i][3], -points2D[i][1]*points3D[i][0], -points2D[i][1]*points3D[i][1], -points2D[i][1]*points3D[i][2], -points2D[i][1]*points3D[i][3]]])
        A[j] = x
        A[j+1] = y
        j+= 2

    ATA = np.dot(A.T, A)
    U, S, V = np.linalg.svd(ATA, full_matrices=True)
    sol = V[:,11]

    a1 = np.array([sol[0], sol[1], sol[2]])
    a2 = np.array([sol[4], sol[5], sol[6]])
    a3 = np.array([sol[8], sol[9], sol[10]])
    b = np.matrix([[sol[3]], [sol[7]], [sol[11]]])

    ro = 1/np.linalg.norm(a3)
    u0 = ro**2*(np.dot(a1, a3))
    v0 = ro**2*(np.dot(a2, a3))
    alfav = math.sqrt(ro**2*np.dot(a2,a2)-v0**2)
    s = (ro**4/alfav)*(np.dot(np.cross(a1, a3), np.cross(a2, a3)))
    alfau = math.sqrt(ro**2*np.dot(a1, a1)-s**2-u0**2)

    K = np.matrix([[alfau, s, u0],[0.0, alfav, v0],[0.0, 0.0, 1]])

    r1 = np.cross(a2,a3)/np.linalg.norm(np.cross(a2,a3))
    r3 = a3
    r2 = np.cross(r3, r1)
    eps = np.sign(b[2][0])
    Kinverse = np.linalg.inv(K)
    T = (np.dot(Kinverse,b))
    T *= eps*ro
    e_parameters = np.matrix([[r1[0], r1[1], r1[2], T[0][0]],[r2[0], r2[1], r2[2], T[1][0]],[r3[0], r3[1], r3[2], T[2][0]]])

    print("Extrinsic_parameters", e_parameters)
    print("Intrinsic parameters", K)

    M = np.dot(K, e_parameters)

    return M 

def calculateMSE(points3D, points2D, M):

    points2DH = []
    points2D = []
    points2D_ref = []

    MSEx = 0
    MSEy = 0

    f = open("Noise_1_3D.txt", "r")
    for line in f:
        pointsDH3 = np.array([[float(line.split()[0]), float(line.split()[1]), float(line.split()[2]), 1.0]])
        a = (M*pointsDH3.T)
        points2DH.append(a)
    f.close()

    g = open("Noise_1_2D.txt", "r")
    for line in g:
        points2D_ref.append([float(line.split()[0]), float(line.split()[1])])
    g.close() 

    for point in points2DH:
        point[0] = point[0]/(point[2])
        point[1] = point[1]/(point[2])
        point[2] = point[2]/(point[2])
        point = np.delete(point, (2), axis = 0)
        points2D.append(point)

    for i in range(0, len(points2D_ref)):
        MSEx += (points2D_ref[i][0]-points2D[i][0])**2
        MSEy += (points2D_ref[i][1]-points2D[i][1])**2

    MSEx = MSEx/len(points2D_ref)
    MSEy = MSEy/len(points2D_ref)
    MSE = MSEx+MSEy

    return MSE 

def RANSAC(points3D, points2D, n, d, k):
    Ms = []
    MSEs= []
    inliers3D = []
    inliers2D = []

    for i in range (0, k):
        points3D_random = []
        points2D_random = []
        distances = []

        for i in range(0, n):
            random.seed()
            random_int = random.randint(0, len(points3D)-1)
            points3D_random.append(points3D[random_int])
            points2D_random.append(points2D[random_int])

        M = calculaparameters(points3D_random, points2D_random)

        for i in range(0, len(points3D_random)):
            distance = math.sqrt(calculateMSE(points3D_random[i], points2D_random[i], M))
            distances.append(distance)
        distances.sort()
        if len(distances)%2 == 0:
            median = (distances[int(len(distances)/2)]+distances[(int(len(distances)/2)-1)])/2
        else:
            median = distances[int((len(distances)/2)-0.5)]
        t = 1.5*median

        for i in range(0, len(points3D)):
            distance = math.sqrt(calculateMSE(points3D[i], points2D[i], M))
            if distance < t:
                inliers3D.append(points3D[i])
                inliers2D.append(points2D[i])

        if len(inliers3D) >= d:
            M = calculaparameters(inliers3D, inliers2D)
            MSE = calculateMSE(inliers3D, inliers2D, M)
            print("Projection Matrix", M)
            print("Mean square error", MSE)

        Ms.append(M)
        MSEs.append(MSE)


    MSEmin = MSE[0]
    Mdef = M[0]
    for i in range(0, len(MSEs)):
        if MSEs[i] < MSEmin:
            MSEmin = MSE[i]
            Mdef = Ms[i]

    return Mdef, MSEmin 

def main():
    f = open("Noise_1_3D.txt", "r")
    g = open("Noise_1_2D.txt", "r")
    points3D = []
    points2D = []

    for line in f:
        points3D.append([float(line.split()[0]), float(line.split()[1]), float(line.split()[2]), 1.0])
    f.close()

    for line in g:
        points2D.append([float(line.split()[0]), float(line.split()[1])])
    g.close()

    Mdef, MSEmin = RANSAC(points3D, points2D, 9, 7, 4)

    M = projection_matrix_calculation(points3D, points2D)
    Mean_squared_error = calculateMSE(points3D, points2D, M)
    print("Projection Matrix", M)
    print("Mean squared error", Mean_squared_error)

    print("Projection Matrix from Ransac", Mdef)
    print("Mean squared error from RANSAC", MSEmin)

if __name__ == '__main__':
    main()

Extrinsic_parameters [[-0.226820510937175 -0.5857591345418667 0.7780994101777498
  matrix([[11.42030511]])]
 [0.013974863594509395 0.03977715509269972 0.034018298042887085
  matrix([[0.26762043]])]
 [-0.05087710973636735 0.018589880863503022 -0.0008363706372929406
  matrix([[0.04604914]])]]
Intrinsic parameters [[ 2.46444560e-02 -1.54714173e-02 -8.03518706e-02]
 [ 0.00000000e+00  1.73881226e+01  3.74472446e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Extrinsic_parameters [[-0.041039502058626914 0.11641595470651797 0.9923522987127857
  matrix([[58.04298173]])]
 [-0.023665370319804233 0.0611054484291752 -0.008147171262763497
  matrix([[0.18648067]])]
 [-0.06158659293327971 -0.02381874048855723 0.00024728950266084734
  matrix([[0.03790295]])]]
Intrinsic parameters [[ 2.53310499e-03  3.23702635e-03 -5.98718997e-02]
 [ 0.00000000e+00  1.29044606e+01 -7.56956151e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Projection Matrix [[0.0035067435219012537 0.001918767024946823